In [6]:
import pandas as pd
import yfinance as yf
import datetime as dt
import pandas_datareader.data as pdr
import numpy as np

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [5]:
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=3aa3b343b4ed8d8c6ac311aec19c377a0a7e331409e803e295eb6a1b433933ad
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [8]:
#Funcoes

def listaAcoes(url):
    dados = pd.read_html(url)
    aux = str(url)
    nome = aux[72:76]
     
    if nome == 'IEEX':
        nome = nome[0:3]
    
    df = dados[0]
    
    unicos_codigos = df['Código'].unique()
    verificacao = len(df['Código']) == len(unicos_codigos)

    if verificacao == True:
        print('Lista de açōes da carteira: ' + ' ' + nome + ' ' +'OK')
    else:
        print('Não foi possivel baixar')
    return df

def yahoo(symbols,S_Day, S_Month, S_Year, E_Day, E_Month, E_Year):
    yf.pdr_override()       
    stocks_hist = pdr.get_data_yahoo(symbols, start=dt.datetime(S_Year, S_Month, S_Day), end=dt.datetime(E_Year, E_Month, E_Day),as_panel = False)
    return stocks_hist

def importacao_preco(Stocks,Start_Day):

    today = dt.date.today()
    E_Day = today.day; E_Month = today.month; E_Year = today.year
    day_before =  dt.datetime.strptime(Start_Day,'%d.%m.%Y')
    
    S_Day = day_before.day; S_Month = day_before.month; S_Year = day_before.year  
    stocks_all=yahoo(Stocks,S_Day, S_Month, S_Year, E_Day, E_Month, E_Year)

    stocks_close = stocks_all['Adj Close']
    #stocks_vol = stocks_all['Volume']
    return stocks_close


def importacao_volume(symbols,Start_Day):
    Stocks=[]
    for symbol in symbols:
        if len(symbol)==5:
            Stocks.append(symbol)
    Stocks=[Stocks[k]+'.SA' for k in range(len(Stocks))]


    today = dt.date.today()
    E_Day = today.day; E_Month = today.month; E_Year = today.year
    day_before =  dt.datetime.strptime(Start_Day,'%d.%m.%Y')

    S_Day = day_before.day; S_Month = day_before.month; S_Year = day_before.year
    stocks_all=yahoo(Stocks,S_Day, S_Month, S_Year, E_Day, E_Month, E_Year)

    stocks_close=stocks_all['Volume']
    return stocks_close

def melhores_volumes(vol,vol_1,vol_2,n):
    media_vol = pd.DataFrame(vol.mean(), columns=['Média de Volume de Tick'])
    media_vol_1 = pd.DataFrame(vol_1.mean(), columns=['Média de Volume de Tick'])
    media_vol_2 = pd.DataFrame(vol_2.mean(), columns=['Média de Volume de Tick'])
    media_total = pd.concat([media_vol,media_vol_1],ignore_index=False)
    media_total = pd.concat([media_total,media_vol_2],ignore_index=False)
    media_total.sort_values(['Média de Volume de Tick'],ascending = False, inplace = True)
    maiores_vol = media_total.head(n)
    return maiores_vol
def tabela_info(preco):
    data = pd.DataFrame()
    for i in range(len(preco.columns)):
        aux = pd.DataFrame()
        aux = preco[preco.columns[i]]
        aux = pd.DataFrame(aux, columns = [preco.columns[i]]).round(4)
        aux['Retorno' + ' ' + preco.columns[i][0:5]+ ' '+ '[%]'] = modelagem.retorno(aux[preco.columns[i]]).round(4)
        aux['Risco' + ' ' + preco.columns[i][0:5]+ ' '+ '[%]'] = modelagem.risco(aux[preco.columns[i]]).round(4)
        data = pd.concat([data,aux], axis = 1)
    return data

class modelagem(object):
    def volatilidade(output):
        stock_return = output.pct_change()
        vol=100*stock_return.rolling(12).std() * np.sqrt(12)
        return vol
        pass
    def retorno(output):
        r1=np.log(output)- np.log(output.shift(1))
        r1 = r1*100
        return r1
        pass
    def risco(output):
        risco = output.std()
        return risco
        pass
    def index_shape(retorno,risco,risco_ref):
        sharpe = (retorno.mean()-risco_ref)/risco
        return sharpe
        pass
        

In [9]:
# Link da b3 com as informacoes
url_IFNC = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=IFNC&idioma=pt-br')
url_BDRX = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=BDRX&idioma=pt-br')
url_ICON = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=ICON&idioma=pt-br')
url_IEE  = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=IEEX&idioma=pt-br')
url_IFIX = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=IFIX&idioma=pt-br')
url_IMAT = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=IMAT&idioma=pt-br')
url_IDIV = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=IDIV&idioma=pt-br')
url_INDX = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=INDX&idioma=pt-br')
url_IMOB = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=IMOB&idioma=pt-br')
url_MLCX = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=MLCX&idioma=pt-br')
url_SMLL = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=SMLL&idioma=pt-br')
url_UTIL = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=UTIL&idioma=pt-br')
url_IVBX = ('http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraTeorica.aspx?Indice=IVBX&idioma=pt-br')

In [10]:
codigo_IFNC =listaAcoes(url_IFNC)
codigo_BDRX =listaAcoes(url_BDRX)
codigo_ICON =listaAcoes(url_ICON)
codigo_IEE  =listaAcoes(url_IEE)
codigo_IFIX =listaAcoes(url_IFIX)
codigo_IMAT =listaAcoes(url_IMAT)
codigo_IDIV =listaAcoes(url_IDIV)
codigo_INDX =listaAcoes(url_INDX)
codigo_IMOB =listaAcoes(url_IMOB)
codigo_MLCX =listaAcoes(url_MLCX)
codigo_SMLL =listaAcoes(url_SMLL)
codigo_UTIL =listaAcoes(url_UTIL)
codigo_IVBX =listaAcoes(url_IVBX)

Lista de açōes da carteira:  IFNC OK
Lista de açōes da carteira:  BDRX OK
Lista de açōes da carteira:  ICON OK
Lista de açōes da carteira:  IEE OK
Lista de açōes da carteira:  IFIX OK
Lista de açōes da carteira:  IMAT OK
Lista de açōes da carteira:  IDIV OK
Lista de açōes da carteira:  INDX OK
Lista de açōes da carteira:  IMOB OK
Lista de açōes da carteira:  MLCX OK
Lista de açōes da carteira:  SMLL OK
Lista de açōes da carteira:  UTIL OK
Lista de açōes da carteira:  IVBX OK


In [11]:
Start_Day = '01.01.2017'

In [12]:
volume_IFNC = importacao_volume(codigo_IFNC['Código'],Start_Day)
#volume_BDRX = importacao_volume(codigo_BDRX['Código'],Start_Day)
volume_ICON = importacao_volume(codigo_ICON['Código'],Start_Day)
volume_IEE  = importacao_volume(codigo_IEE['Código'],Start_Day)
#volume_IFIX = importacao_volume(codigo_IFIX['Código'],Start_Day)
#volume_IMAT = importacao_volume(codigo_IMAT['Código'],Start_Day)
#volume_IDIV = importacao_volume(codigo_IDIV['Código'],Start_Day)
#volume_IMOB = importacao_volume(codigo_IMOB['Código'],Start_Day)
#volume_MLCX = importacao_volume(codigo_MLCX['Código'],Start_Day)
#volume_SMLL = importacao_volume(codigo_SMLL['Código'],Start_Day)
#volume_UTIL = importacao_volume(codigo_UTIL['Código'],Start_Day)
#volume_IVBX = importacao_volume(codigo_IVBX['Código'],Start_Day)


[*********************100%***********************]  16 of 16 completed
[*********************100%***********************]  61 of 61 completed
[*********************100%***********************]  14 of 14 completed


In [13]:
n = 50 
maiores_vol = melhores_volumes(volume_IFNC,volume_ICON,volume_IEE,n)
maiores_vol

,Média de Volume de Tick
VVAR3.SA,2.570649e+07
ITUB4.SA,2.285454e+07
ITSA4.SA,2.205189e+07
BBDC4.SA,2.064184e+07
COGN3.SA,1.946320e+07
ABEV3.SA,1.941527e+07
CIEL3.SA,1.581139e+07
JBSS3.SA,1.228174e+07
BBAS3.SA,1.190227e+07
MGLU3.SA,1.182193e+07


In [14]:
preco = importacao_preco(list(maiores_vol.index),Start_Day)

[*********************100%***********************]  50 of 50 completed


In [18]:
preco

,ABEV3.SA,AMAR3.SA,B3SA3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BEEF3.SA,BIDI4.SA,BMGB4.SA,BPAN4.SA,BRFS3.SA,BTOW3.SA,CEAB3.SA,CIEL3.SA,CMIG4.SA,COGN3.SA,CPFE3.SA,CRFB3.SA,CVCB3.SA,CYRE3.SA,ELET3.SA,ENBR3.SA,EQTL3.SA,FLRY3.SA,GFSA3.SA,GNDI3.SA,HAPV3.SA,HYPE3.SA,IRBR3.SA,ITSA4.SA,ITUB4.SA,JBSS3.SA,JHSF3.SA,LAME4.SA,LIGT3.SA,LREN3.SA,MEAL3.SA,MGLU3.SA,MRFG3.SA,MRVE3.SA,NEOE3.SA,NTCO3.SA,QUAL3.SA,RADL3.SA,RENT3.SA,TCSA3.SA,VIVA3.SA,VVAR3.SA,YDUQ3.SA
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,14.968172,6.38,14.866173,23.298105,17.502491,16.620461,19.753111,11.379594,NaN,NaN,1.210790,47.900002,9.846180,NaN,18.427422,6.107166,12.361406,21.605221,NaN,22.343370,8.550741,19.634739,11.340909,9.815331,15.143176,13.4518,NaN,NaN,22.895599,NaN,5.960370,18.684441,10.949058,1.347535,16.171041,17.256519,18.077227,4.839031,1.069914,6.400000,7.887719,NaN,NaN,16.155283,10.482961,10.025175,2.22555,NaN,3.781286,13.660166
2017-01-03,15.271022,6.44,15.477986,24.364025,18.216722,17.329597,20.456043,11.656956,NaN,NaN,1.163861,49.500000,10.097200,NaN,18.941414,6.241648,12.974829,21.656540,NaN,23.013090,9.274393,19.495062,11.383227,9.795611,15.064622,14.8508,NaN,NaN,23.534262,NaN,6.221735,19.475744,11.094398,1.403682,16.433512,17.816668,18.631811,4.998670,1.066656,6.460000,8.066114,NaN,NaN,16.859179,10.884090,10.446912,2.33484,NaN,3.701680,14.420538
2017-01-04,15.151719,6.27,15.770195,24.237139,18.210630,17.219801,21.102173,11.398752,NaN,NaN,1.154474,48.910000,10.328800,NaN,18.758993,6.067615,12.900474,21.605221,NaN,23.741045,9.615427,19.713221,11.450933,9.833259,15.400655,15.3888,NaN,NaN,23.674246,NaN,6.258029,19.559345,11.094398,1.459830,16.404341,17.620125,18.467808,5.138352,1.009848,6.600000,8.066114,NaN,NaN,17.005110,10.730209,10.613367,2.39445,NaN,3.771335,14.349806
2017-01-05,15.234314,6.32,15.578434,24.177917,18.313978,17.410454,20.676153,11.283996,NaN,NaN,1.154474,48.529999,10.637700,NaN,18.555162,5.972685,12.733177,21.562454,NaN,24.459295,9.490658,19.634739,11.425543,9.815331,15.710502,15.8193,NaN,NaN,23.656748,NaN,6.316109,19.826818,11.065331,1.553408,16.447870,17.492371,18.295959,5.347878,1.044560,6.580000,8.101842,NaN,NaN,16.902102,10.569409,10.452703,2.48387,NaN,3.781286,14.119926
2017-01-06,15.105830,6.29,15.569300,24.440168,18.453777,17.520151,20.356640,11.236149,NaN,NaN,1.163861,46.220001,10.309500,NaN,18.595928,5.893574,12.640232,21.579563,NaN,24.255466,9.432432,19.608549,11.129325,9.770512,15.710502,15.4964,NaN,NaN,23.473022,NaN,6.294332,19.893692,11.142846,1.562766,16.408880,17.482546,18.084976,5.268059,1.037828,6.330000,8.108988,NaN,NaN,16.730419,10.591887,10.449759,2.44413,NaN,3.741483,14.013827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-15,12.890000,8.14,57.279999,31.990000,18.920000,20.480000,26.100000,13.840000,19.969999,5.14,8.640000,21.610001,99.900002,12.90,4.600000,10.620000,5.800000,29.700001,20.920000,17.480000,24.459999,33.099998,18.600000,22.780001,29.170000,4.6500,71.269997,66.050003,31.639999,5.92,9.260000,23.730000,24.100000,7.500000,31.260000,14.820000,44.259998,3.690000,89.580002,17.110001,17.620001,17.900000,53.500000,32.959999,22.796000,52.820000,10.74000,25.49,18.549999,29.700001
2020-09-16,12.560000,8.59,57.020000,32.090000,19.040001,2052.000000,26.219999,13.340000,20.059999,5.13,8.530000,21.350000,98.500000,13.07,4.580000,10.810000,5.830000,29.950001,21.049999,18.200001,24.010000,33.380001,18.570000,23.139999,29.700001,4.5400,71.860001,66.239998,30.860001,5.92,9.260000,23.660000,23.340000,7.590000,30.879999,15.180000,44.759998,3.680000,89.169998,16.700001,17.490000,17.900000,52.730000,33.080002,22.862000,52.680000,10.78000,24.92,18.160000,30.400000
2020-09-17,13.160000,8.27,56.680000,32.150002,19.110001,2057.000000,26.100000,13.160000,19.629999,5.08,8.710000,21.360001,97.309998,13.22,4.560000,10.920000,5.770000,30.010000,21

In [26]:
tabela_info(preco)

AttributeError: ignored

In [27]:
data = pd.DataFrame(dados.mean(), columns = ['Informaces das medias'])
data

NameError: ignored